In [1]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [1]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
from nltk.metrics import accuracy
import re


In [2]:
 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
 

In [3]:
news=pd.read_csv('FakeNewsDataset.csv')

In [4]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
news.shape 

(20800, 5)

In [6]:
news.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
news=news.fillna(' ')

In [8]:
news.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
news['MainContent']=news['title']+" "+news['author']

In [10]:
news['MainContent']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799              What Keeps the F-35 Alive David Swanson
Name: MainContent, Length: 20800, dtype: object

In [11]:
ps=PorterStemmer()
stopword=set(stopwords.words('english'))
def stemming(content):
    temp=re.sub('[^a-zA-Z]',' ',content)
    temp=re.sub(' +',' ',temp)
    temp=temp.lower()
    temp=temp.split()    
    temp=[ps.stem(word) for word in temp if word not in stopword]
    temp=' '.join(temp)
    return temp

In [12]:
news['MainContent']=news['MainContent'].apply(stemming)

In [13]:
news['MainContent']

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: MainContent, Length: 20800, dtype: object

In [14]:
X=news['MainContent'].values
y=news['label'].values

In [15]:
print(X)

['hous dem aid even see comey letter jason chaffetz tweet darrel lucu'
 'flynn hillari clinton big woman campu breitbart daniel j flynn'
 'truth might get fire consortiumnew com' ...
 'maci said receiv takeov approach hudson bay new york time michael j de la merc rachel abram'
 'nato russia hold parallel exercis balkan alex ansari'
 'keep f aliv david swanson']


In [16]:
vector=TfidfVectorizer()
 
X=vector.fit_transform(X)

In [17]:
X[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12 stored elements and shape (1, 17128)>

#SPLITTING DATA

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,stratify=y,random_state=42)


In [19]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [20]:
models=[LogisticRegression(),SVC(kernel='linear'),KNeighborsClassifier(),RandomForestClassifier()]


BEST MODEL SELECTION : CROSS VALIDATION  , HYPERPARAMETERTUNING

#*CROSS VALIDATION

In [25]:
""" def compare_models():
    for model in models: 
        cv_score_model=cross_val_score(model,X,y,cv=5)
        mean_accuracy=sum(cv_score_model)/len(cv_score_model)
        mean_accuracy=round(mean_accuracy*100,2)
        print("cross val accuracy ",model," = ", cv_score_model)
        print("accuracy % of ",model,mean_accuracy)
        print("-------------------------------------------") """

In [ ]:
""" compare_models() """

cross val accuracy  LogisticRegression()  =  [0.97596154 0.97427885 0.97908654 0.97620192 0.97620192]
accuracy % of  LogisticRegression() 97.63
-------------------------------------------
cross val accuracy  SVC(kernel='linear')  =  [0.99350962 0.98990385 0.99399038 0.99350962 0.98990385]
accuracy % of  SVC(kernel='linear') 99.22
-------------------------------------------
cross val accuracy  KNeighborsClassifier()  =  [0.52620192 0.52331731 0.52139423 0.52235577 0.52379808]
accuracy % of  KNeighborsClassifier() 52.34
-------------------------------------------
cross val accuracy  RandomForestClassifier()  =  [0.99423077 0.99182692 0.99543269 0.99399038 0.99278846]
accuracy % of  RandomForestClassifier() 99.37
-------------------------------------------


SO NOW WE CAN SAY THAT RANDOM FOREST IS THE BEST FOR OUR DATASET 


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=5,criterion='gini',max_depth=4)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=4, n_estimators=5)

In [48]:
from sklearn.metrics import accuracy_score


In [49]:
y_pred=model.predict(X_test)
print("accuracy : ",accuracy_score(y_test,y_pred))

accuracy :  0.9211829836829837


NOW LETS TUNE THE HYPERPARAMETERS TO GET MORE EFFICIENY 

In [27]:
""" from sklearn.model_selection import GridSearchCV """

THIS GRID SEARCH IS USED TO GET THE BEST HYPERPARAMETERS FOR ANY MODEL 

In [ ]:
"""  
parameters={
    'n_estimators':[5,10,20,50,100],
    'max_depth':[2,5,10,15,20],
    'criterion':['gini','log_loss','entropy'] 
}
classifier=GridSearchCV(model,parameters,cv=5)  
 
classifier.fit(X,y) """

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_depth=4, n_estimators=5),
             param_grid={'criterion': ['gini', 'log_loss', 'entropy'],
                         'max_depth': [2, 5, 10, 15, 20],
                         'n_estimators': [5, 10, 20, 50, 100]})

In [29]:
""" classifier.best_params_ """

{'criterion': 'gini', 'max_depth': 20, 'n_estimators': 100}

In [36]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=20)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("accuracy : ",accuracy_score(y_test,y_pred))

accuracy :  0.9211829836829837


In [30]:
""" best_model=classifier.best_estimator_    """

In [ ]:
""" y_pred=best_model.predict(X_test)
print("accuracy : ",accuracy_score(y_test,y_pred)) """

accuracy :  0.9562937062937062


In [32]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [33]:
print("accuracy : ",accuracy_score(y_test,y_pred))

accuracy :  0.9562937062937062


In [34]:
print(confusion_matrix(y_test,y_pred))

[[3136  292]
 [   8 3428]]


In [35]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95      3428
           1       0.92      1.00      0.96      3436

    accuracy                           0.96      6864
   macro avg       0.96      0.96      0.96      6864
weighted avg       0.96      0.96      0.96      6864

